Simulation of variable weight constant liquidity relay, in which stakers can stake assets with custom weightedness, and the contract will maintain the individual ratios of all stakers, using:
1. A self-adjusting global relay weightedness (important for ensuring swaps are executed correctly)
2. Issuance of n-sets of liquidity tokens for an n-asset relay

This simulation uses and ETH:token relay (analagous to ETH:DAI). It is a 2-asset relay, and therefore will use 2 sets of liqiuidity tokens, minted and distributed to stakers. 

Something to consider: floating point and logarithmic calculations are a challenge to emulate in EVM. 

In [219]:
#In this version, liquidity tokens ($_ETH_lt, $_token_lt) are issued directly prop. to ETH and tokens staked

e = 2.7182818 #Euler's constant, for log operations

ETH_balance = 10000 #initial ETH balance
ETH_weight = 0.9090909090909091 #initial normalized weight of ETH (currentl @ 10:1 ETH:token)
token_balance = 190000 #initial token balance (chosen to emulate current ETH:DAI price, arbitrarily)
token_weight = 0.09090909090909091 #initial normalized weight of tokens

V = ETH_balance^ETH_weight * token_balance^token_weight #formula from Balancer whitepaper
SP = (token_balance/token_weight) / (ETH_balance/ETH_weight) #spot price

ETH_lt = ETH_balance #initial ETH liquidity tokens minted
token_lt = token_balance #initial token liquidity tokens minted

println("V: ", V)
println("SP: ", SP)
println("ETH liq. tokens minted: ", ETH_lt)
println("Token liq. tokens minted: ", token_lt)

V: 13069.239821127008
SP: 190.0
ETH liq. tokens minted: 10000
Token liq. tokens minted: 190000


In [220]:
#add stake with CUSTOM weighting — staker A
#V should change, price (SP) should remain the same
#mint new ETH_lt, token_LT

A_weight = (0.7,0.3) #custom weights (ETH,token)
A_ETH_stake = 50 #how much ETH to stake
A_token_stake = (A_ETH_stake*SP) * (A_weight[2]/A_weight[1]) #how many tokens to stake, acc. to weights
ETH_balance += A_ETH_stake #add stake to ETH pool
token_balance += A_token_stake #add token stake to token pool

ETH_weight = 1 - (1/((ETH_balance * SP / token_balance )+1)) #update ETH weight (weighted mean)
token_weight = 1/((ETH_balance * SP / token_balance )+1) #update token weight (weighted mean)

V = ETH_balance^ETH_weight * token_balance^token_weight #update invariant (it should change)

ETH_lt += A_ETH_stake #update ETH liq. token pool
token_lt += A_token_stake #update token liq. token pool

A_ETH_lt = A_ETH_stake #how many liq. tokens go to staker A
A_token_lt = A_token_stake #how many liq. tokens go to staker A

A_stake = (A_token_lt/token_lt * token_balance) + ((A_ETH_lt/ETH_lt * ETH_balance)*SP) #cumulative value of original stake

println("ETH_weight: ", ETH_weight) #updated weight
println("token_weight: ", token_weight) #updated weight
println()
println("ETH liq. tokens: ", A_ETH_lt) #staker's ETH liq. tokens
println("token liq. tokens: ", A_token_lt) #staker's token liq. tokens
println("Stake value: ", A_stake)
println()

#check spot price (should be same)
SP = (token_balance/token_weight) / (ETH_balance/ETH_weight) 
println("Spot price: ", SP)

ETH_weight: 0.9077419354838709
token_weight: 0.09225806451612904

ETH liq. tokens: 50
token liq. tokens: 4071.4285714285716
Stake value: 13571.428571428572

Spot price: 190.0


In [221]:
#add another stake with CUSTOM weighting (same as above, just a new stake) — staker B
#V should change, price should remain the same
#mint new ETH_lt, token_LT

B_weight = (0.4, 0.6) #custom weights (ETH,token)
B_ETH_stake = 300 #how much ETH to stake
B_token_stake = (B_ETH_stake*SP) * (B_weight[2]/B_weight[1])
ETH_balance += B_ETH_stake #update ETH liquidity pool
token_balance += B_token_stake #update token liquidity pool

ETH_weight = 1 - (1/((ETH_balance * SP / token_balance )+1)) #update ETH weight
token_weight = 1/((ETH_balance * SP / token_balance )+1) #update token weight

V = ETH_balance^ETH_weight * token_balance^token_weight #update invariant

ETH_lt += B_ETH_stake
token_lt += B_token_stake

B_ETH_lt = B_ETH_stake #new staker's ETH liq. tokens
B_token_lt = B_token_stake #staker's token liq. tokens
B_stake = (B_token_lt/token_lt * token_balance) + ((B_ETH_lt/ETH_lt * ETH_balance)*SP) #cumulative value of original stake

println("ETH_weight: ", ETH_weight)
println("token_weight: ", token_weight)
println()
println("ETH liq. tokens: ", B_ETH_lt)
println("token liq. tokens: ", B_token_lt)
println("Stake value: ", B_stake)
println()

#check spot price (should not change from staking)
SP = (token_balance/token_weight) / (ETH_balance/ETH_weight) 
println("Spot price: ", SP)

ETH_weight: 0.8755287009063444
token_weight: 0.12447129909365559

ETH liq. tokens: 300
token liq. tokens: 85499.99999999999
Stake value: 142500.0

Spot price: 190.0


In [222]:
#Verify that staker A still has originally-weighted assets and balances (see above for values)
show(A_ETH_lt/ETH_lt * ETH_balance == A_ETH_stake)
println()
show(A_token_lt/token_lt * token_balance == A_token_stake)
println()
show(round((A_token_lt/token_lt * token_balance) / ((A_ETH_lt/ETH_lt * ETH_balance)*SP), digits = 5) ==
    round(A_weight[2]/A_weight[1], digits = 5))

true
true
true

In [223]:
#add another stake with CUSTOM weighting — staker C
#V should change, price should remain the same
#mint new ETH_lt, token_LT

C_weight = (0.2, 0.8) #custom weights (ETH,token)
C_ETH_stake = 70 #how much ETH to stake
C_token_stake = (C_ETH_stake*SP) * (C_weight[2]/C_weight[1])
ETH_balance += C_ETH_stake #update ETH liquidity pool
token_balance += C_token_stake #update token liquidity pool

ETH_weight = 1 - (1/((ETH_balance * SP / token_balance )+1)) #update ETH weight
token_weight = 1/((ETH_balance * SP / token_balance )+1) #update token weight

V = ETH_balance^ETH_weight * token_balance^token_weight #update invariant

ETH_lt += C_ETH_stake
token_lt += C_token_stake

C_ETH_lt = C_ETH_stake #new staker's ETH liq. tokens
C_token_lt = C_token_stake #new staker's token liq. tokens
C_stake = (C_token_lt/token_lt * token_balance) + ((C_ETH_lt/ETH_lt * ETH_balance)*SP) #cumulative value of original stake

println("ETH_weight: ", ETH_weight)
println("token_weight: ", token_weight)
println()
println("ETH liq. tokens: ", your_ETH_lt)
println("token liq. tokens: ", your_token_lt)
println("Stake value: ", C_stake)
println()

#check spot price (should not change from staking)
SP = (token_balance/token_weight) / (ETH_balance/ETH_weight) 
println("Spot price: ", SP)

ETH_weight: 0.8561032863849765
token_weight: 0.14389671361502349

ETH liq. tokens: 300
token liq. tokens: 76949.99999999997
Stake value: 66500.0

Spot price: 190.0


In [224]:
#Verify that staker B still has originally-weighted assets and balances (see above for values)
show(B_ETH_lt/ETH_lt * ETH_balance == B_ETH_stake)
println()
show(B_token_lt/token_lt * token_balance == B_token_stake)
println()
show(round((B_token_lt/token_lt * token_balance) / ((B_ETH_lt/ETH_lt * ETH_balance)*SP), digits = 5) ==
    round(B_weight[2]/B_weight[1], digits = 5))

true
true
true

In [225]:
#Does it work with swaps?

#sell tokens (buy ETH)

V = ETH_balance^ETH_weight * token_balance^token_weight #update invariant (should not change from swap)

sale = 7000 #how many tokens to send to contract (swap for ETH)
token_balance += sale #add sale tokens to contract
old_ETH_balance = ETH_balance #store for calculation below
ETH_balance = e^((log(V / (token_balance^token_weight)))/ETH_weight) #update ETH balance according to sale
ETH_received = old_ETH_balance - ETH_balance #ETH received for swapped tokens
println("Tokens sold: ", sale)
println("ETH received: ", ETH_received)
println()

V = ETH_balance^ETH_weight * token_balance^token_weight #update invariant (should not chance)
SP = (token_balance/token_weight) / (ETH_balance/ETH_weight) 
println("V: ", V)
println("SP: ", SP)

Tokens sold: 7000
ETH received: 36.39725206026378

V: 17152.537933040985
SP: 194.676746822169


In [226]:
#Verify that staker A,B,C still have originally-weighted assets 
#balances will have changed b/c of swap, but weithed ratio should remain constant

println(round((A_token_lt/token_lt * token_balance) / ((A_ETH_lt/ETH_lt * ETH_balance)*SP), digits = 5) ==
    round(A_weight[2]/A_weight[1], digits = 5))

println(round(((B_ETH_lt/ETH_lt * ETH_balance)*SP) / (B_token_lt/token_lt * token_balance), digits = 5) ==
    round(B_weight[1]/B_weight[2], digits = 5))

println(round(((C_ETH_lt/ETH_lt * ETH_balance)*SP) / (C_token_lt/token_lt * token_balance), digits = 5) ==
    round(C_weight[1]/C_weight[2], digits = 5))

true
true
true


In [227]:
#sell ETH (buy tokens)
V = ETH_balance^ETH_weight * token_balance^token_weight #update invariant (should not change from swap)

sale = 20 #how much ETH sent to contract (swapped for tokens)
old_token_balance = token_balance #store for calculation below
ETH_balance += sale 
token_balance = e^((log(V / (ETH_balance^ETH_weight)))/token_weight) #update token balance according to sale
tokens_received = old_token_balance - token_balance
println("ETH sold: ", sale)
println("tokens received: ", tokens_received)
println()

V = ETH_balance^ETH_weight * token_balance^token_weight
SP = (token_balance/token_weight) / (ETH_balance/ETH_weight) 
println("V: ", V)
println("SP: ", SP)

ETH sold: 20
tokens received: 3867.6538681865786

V: 17152.5376042283
SP: 192.09073235980034


In [228]:
#Verify that staker A,B,C still have originally-weighted assets 
#balances will have changed b/c of swap, but weithed ratio should remain constant

println(round((A_token_lt/token_lt * token_balance) / ((A_ETH_lt/ETH_lt * ETH_balance)*SP), digits = 5) ==
    round(A_weight[2]/A_weight[1], digits = 5))

println(round(((B_ETH_lt/ETH_lt * ETH_balance)*SP) / (B_token_lt/token_lt * token_balance), digits = 5) ==
    round(B_weight[1]/B_weight[2], digits = 5))

println(round(((C_ETH_lt/ETH_lt * ETH_balance)*SP) / (C_token_lt/token_lt * token_balance), digits = 5) ==
    round(C_weight[1]/C_weight[2], digits = 5))

true
true
true


In [234]:
#Cumulative value of stakes A,B,C after stakes and swaps, relative to original
#**In absolute terms**

println("Current relay price: ", SP)
print("Change in value of stake A: ")
println((A_token_lt/token_lt * token_balance) + ((A_ETH_lt/ETH_lt * ETH_balance)*SP) - A_stake)
print("Change in value of stake B: ")
println((B_token_lt/token_lt * token_balance) + ((B_ETH_lt/ETH_lt * ETH_balance)*SP) - B_stake)
print("Change in value of stake C: ")
println((C_token_lt/token_lt * token_balance) + ((C_ETH_lt/ETH_lt * ETH_balance)*SP) - C_stake)

Current relay price: 192.09073235980034
Change in value of stake A: 127.74657960087279
Change in value of stake B: 1341.3390858091589
Change in value of stake C: 625.9582400442887


In [235]:
#Cumulative value of stakes A,B,C after stakes and swaps, relative to original
#**In percentile terms**
#Note: if all stakes were made at same SP, they should have the same percentile change in value

println("Current relay price: ", SP)
print("Change in value of stake A: ")
println(((A_token_lt/token_lt * token_balance) + ((A_ETH_lt/ETH_lt * ETH_balance)*SP) - A_stake)/A_stake)
print("Change in value of stake B: ")
println(((B_token_lt/token_lt * token_balance) + ((B_ETH_lt/ETH_lt * ETH_balance)*SP) - B_stake)/B_stake)
print("Change in value of stake C: ")
println(((C_token_lt/token_lt * token_balance) + ((C_ETH_lt/ETH_lt * ETH_balance)*SP) - C_stake)/C_stake)

Current relay price: 192.09073235980034
Change in value of stake A: 0.009412905865327469
Change in value of stake B: 0.00941290586532743
Change in value of stake C: 0.009412905865327649
